In [1]:
from unsloth import FastLanguageModel
import torch


MODEL = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
                                                     model_name=MODEL,
                                                     max_seq_length=2048,
                                                     dtype=None,
                                                     load_in_4bit=True,
                                                     )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r=4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    use_gradient_checkpointing= "unsloth",
    lora_alpha= 16,
    lora_dropout = 0,
    bias = "none",
    use_rslora= False,
    loftq_config= None
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.1.6 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [3]:
from datasets import load_dataset
from unsloth import to_sharegpt
from unsloth import standardize_sharegpt

dataset = load_dataset("vicgalle/alpaca-gpt4", split = "train")

dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name= "output",
    conversation_extension= 3,
)

dataset = standardize_sharegpt(dataset)

In [11]:
import os
print(os.environ.get("CUDA_VISIBLE_DEVICES", "Not set"))
# If this returns a single GPU index (e.g., "0"), PyTorch will only see 1 GPU.

1


In [12]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

device = "cuda:1"
model = model.to(device)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(model=model,
                    args=TrainingArguments(
                        output_dir="output",
                        overwrite_output_dir=True,
                        per_device_train_batch_size=2,
                        gradient_accumulation_steps=4,
                        num_train_epochs=1,
                        logging_dir="logs",
                        logging_steps=1,
                        save_steps=1,
                        max_steps=60,
                        learning_rate=2e-4,
                        optim="adamw_8bit",
                        weight_decay=0.01,
                        bf16=True,
                    ),
                    tokenizer=tokenizer,
                    train_dataset=dataset,
                    eval_dataset=None,
                     )



In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 3,407,872
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: usamainam3 (usamaparacha). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/unsloth/models/_utils.py:584: SyntaxWarning: invalid escape sequence '\.'
  source = re.sub("([^\.])nn\.", r"\1torch.nn.", source)
/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/unsloth/models/_utils.py:847: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/unsloth/models/_utils.py:947: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,


OutOfMemoryError: CUDA out of memory. Tried to allocate 496.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 451.75 MiB is free. Process 3723318 has 16.98 GiB memory in use. Including non-PyTorch memory, this process has 6.24 GiB memory in use. Of the allocated memory 5.88 GiB is allocated by PyTorch, and 50.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)